### Linear Regression using built-in Pytorch Functions

In [1]:
import numpy as np
import torch

#### Create Dataset 

For given set of inputs what is the output(target)

In [4]:
inputs = np.array([[73, 67, 43],[91, 88, 64],[87, 134,58],[102, 43,37],[69,96,70],
                  [73, 67, 43],[91, 88, 64],[87, 134,58],[102, 43,37],[69,96,70],
                  [73, 67, 43],[91, 88, 64],[87, 134,58],[102, 43,37],[69,96,70]], dtype='float32')
inputs = torch.from_numpy(inputs)
target = np.array([[56,70], [81,101], [119,133], [22,37], [103,119],
                  [56,70], [81,101], [119,133], [22,37], [103,119],
                  [56,70], [81,101], [119,133], [22,37], [103,119]], dtype='float32')
target = torch.from_numpy(target)


#### Dataset and DataLoad

In [5]:
from torch.utils.data import TensorDataset

We create a dataset from given inputs by slicing a part of inputs instead of using entire dataset.
Fist tensor represents input and second tensor represents target.

In [6]:
train_ds = TensorDataset(inputs, target)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [7]:
from torch.utils.data import DataLoader

DataLoader is used to split the data into batches which also enables shuffling for better model.

In [8]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
train_dl

Dataloader is used in for-in loop.

In [11]:
for x, y in train_dl:
    #print('batch:')
    print(x)
    print(y)
    break

tensor([[ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.]])
tensor([[119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [103., 119.],
        [119., 133.]])


#### Loss function

We use _nn.Linear_ for initializing weights and biases with random numbers with requires_grad set to True

In [12]:
import torch.nn as nn
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.2320, -0.0114, -0.0173],
        [ 0.0987, -0.0019, -0.1276]], requires_grad=True)
Parameter containing:
tensor([-0.5086,  0.0492], requires_grad=True)


.parameters() enlists all the weights and biases.

In [14]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2320, -0.0114, -0.0173],
         [ 0.0987, -0.0019, -0.1276]], requires_grad=True),
 Parameter containing:
 tensor([-0.5086,  0.0492], requires_grad=True)]

we create predictions

In [15]:
predict = model(inputs)
predict

tensor([[14.9228,  1.6413],
        [18.4965,  0.6988],
        [17.1496,  0.9826],
        [22.0289,  5.3146],
        [13.1966, -2.2534],
        [14.9228,  1.6413],
        [18.4965,  0.6988],
        [17.1496,  0.9826],
        [22.0289,  5.3146],
        [13.1966, -2.2534],
        [14.9228,  1.6413],
        [18.4965,  0.6988],
        [17.1496,  0.9826],
        [22.0289,  5.3146],
        [13.1966, -2.2534]], grad_fn=<AddmmBackward>)

In [16]:
import torch.nn.functional as F

We use built-in loss funtion to calculate Mean Square Error. 

In [17]:
loss_fn = F.mse_loss

In [21]:
loss = loss_fn(predict, target)
print(loss)

tensor(7190.0376, grad_fn=<MseLossBackward>)


#### Optimization

Optimization is done to adjust weights and biases that are enlisted with the help of .parameters().
We give learning rate(lr) as well to control amount of modification.
SGD stands for stochastic gradient descent because samples are selected  in batches and not as single group.

In [22]:
opt = torch.optim.SGD(model.parameters(), lr= 1e-5)

optim.SGD knows which matrices require modification in the above update step.

#### Train the model

In [35]:
# Funtion to train model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    # Repeat for a number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches
        for x, y in train_dl:
            
            # generate Predictions
            predict = model(x)
            
            # Calculate loss 
            loss = loss_fn(predict, y)
            
            # compute Gradient
            loss.backward()
            
            # .step() is equivalent to weight -= weight.grad * 1e-5 adn so for bias
            opt.step()
            
            # Reset gradients to zero
            opt.zero_grad()
        
        # Print loss  for every 10th epoch 
        if (epoch+1) % 10 == 0:
            print('Epoch[{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss))

Apply parameters to the fit() 

In [36]:
fit(100, model, loss_fn, opt, train_dl)

Epoch[10/100], Loss: 227.7663
Epoch[20/100], Loss: 132.5481
Epoch[30/100], Loss: 92.4056
Epoch[40/100], Loss: 81.3599
Epoch[50/100], Loss: 43.2501
Epoch[60/100], Loss: 62.0897
Epoch[70/100], Loss: 42.4540
Epoch[80/100], Loss: 34.8602
Epoch[90/100], Loss: 14.0087
Epoch[100/100], Loss: 19.2965


Compare corrected predictions with target.

In [38]:
predict = model(inputs)
predict

tensor([[ 58.1934,  71.4634],
        [ 80.6089,  97.6036],
        [120.5938, 138.1041],
        [ 27.3858,  43.2665],
        [ 95.3665, 110.1630],
        [ 58.1934,  71.4634],
        [ 80.6089,  97.6036],
        [120.5938, 138.1041],
        [ 27.3858,  43.2665],
        [ 95.3665, 110.1630],
        [ 58.1934,  71.4634],
        [ 80.6089,  97.6036],
        [120.5938, 138.1041],
        [ 27.3858,  43.2665],
        [ 95.3665, 110.1630]], grad_fn=<AddmmBackward>)

In [39]:
target

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])